<h1> Building A Traffic Collision Prediction Model </h1>

This notebook will involve data wrangling, feature engineering and building a prediction model using supervised learning to predict the severity of an accident given attributes of the dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [2]:
pd.set_option('display.max_columns',50)
df = pd.read_csv(r'Data-Collisions.csv', parse_dates = ['INCDATE'])
df.head()

C:\Users\ASUS\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,LOCATION,EXCEPTRSNCODE,EXCEPTRSNDESC,SEVERITYCODE.1,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INCDATE,INCDTTM,JUNCTIONTYPE,SDOT_COLCODE,SDOT_COLDESC,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,5TH AVE NE AND NE 103RD ST,,NaN,2,Injury Collision,Angles,2,0,0,2,2013-03-27 00:00:00+00:00,3/27/2013 2:54:00 PM,At Intersection (intersection related),11,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,N,Overcast,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,AURORA BR BETWEEN RAYE ST AND BRIDGE WAY N,NaN,NaN,1,Property Damage Only Collision,Sideswipe,2,0,0,2,2006-12-20 00:00:00+00:00,12/20/2006 6:55:00 PM,Mid-Block (not related to intersection),16,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, LEFT SIDE ...",NaN,0,Raining,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,4TH AVE BETWEEN SENECA ST AND UNIVERSITY ST,NaN,NaN,1,Property Damage Only Collision,Parked Car,4,0,0,3,2004-11-18 00:00:00+00:00,11/18/2004 10:20:00 AM,Mid-Block (not related to intersection),14,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, REAR END",NaN,0,Overcast,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,2ND AVE BETWEEN MARION ST AND MADISON ST,,NaN,1,Property Damage Only Collision,Other,3,0,0,3,2013-03-29 00:00:00+00:00,3/29/2013 9:26:00 AM,Mid-Block (not related to intersection),11,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,N,Clear,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,SWIFT AVE S AND SWIFT AV OFF RP,NaN,NaN,2,Injury Collision,Angles,2,0,0,2,2004-01-28 00:00:00+00:00,1/28/2004 8:04:00 AM,At Intersection (intersection related),11,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,0,Raining,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


<h1> Cleaning the Data </h1>

In this dataset, the columns SEVERITYCODE and SEVERITYCODE.1 are identical to each other and convey the same information. Hence in the following step, it is dropped.

In [3]:
df.drop('SEVERITYCODE.1',axis = 1, inplace = True)

In [4]:
#drop the unique key for incident,secondary key for incident,report no. as they dont contribute to prediction of severity
df.drop(['INCKEY','COLDETKEY','REPORTNO'],axis = 1, inplace = True)

The UNDERIFL column contains 4 different values. Y,1,N,0 and NaN. It is necessary to make the column consistent to have only **1 for Under the Influence and 0 for Not Under the Influence** Filling up the missing values and making consistent binary indications in the Under Influence columns

In [5]:
df['UNDERINFL'].unique()

array(['N', '0', nan, '1', 'Y'], dtype=object)

In [6]:
df.loc[(df['UNDERINFL'] == 'Y'),'UNDERINFL'] = 1
df.loc[(df['UNDERINFL'] == 'N'),'UNDERINFL'] = 0
df.loc[(df['UNDERINFL'] == '1'),'UNDERINFL'] = 1
df.loc[(df['UNDERINFL'] == '0'),'UNDERINFL'] = 0
df['UNDERINFL'].unique()

array([0, nan, 1], dtype=object)

In [7]:
temp = df[['SEVERITYCODE','SEVERITYDESC']]
temp[~(temp.duplicated())]

,SEVERITYCODE,SEVERITYDESC
0,2,Injury Collision
1,1,Property Damage Only Collision


In [34]:
df['UNDERINFL'].isna().value_counts()

False    180067
Name: UNDERINFL, dtype: int64

This dataset has only 2 severity levels

These 2 functions serve to find the percentage of Missing Values in each column and then produce a Dataframe of these results to help up get a better understanding of the missing data.

In [9]:
def missingvaluepercentages(features,datafr):
    numberofnanvalues = []
    nanvaluepercentage = []
    featureswithnan = []
    totalvalues = datafr.shape[0]
    for feature in features:
        try:
            nanvalues = datafr[feature].isna().value_counts()[1]
        except:
            nanvalues = 0
        numberofnanvalues.append(nanvalues)
        if nanvalues > 0:
            nanvaluepercent = ((nanvalues/totalvalues)*100).round(2)
        else:
            nanvaluepercent = 0.00
        nanvaluepercentage.append(nanvaluepercent)
        featureswithnan.append(feature)
            
    return numberofnanvalues,nanvaluepercentage,featureswithnan


def visualizemissingdata(numberofnanvalues,nanvaluepercentage,features):
    #order from highest to lowest for nanvalues and nanvalue%
    idx = np.array(numberofnanvalues).argsort()[::-1]
    nanvaluearray = np.array(numberofnanvalues)[idx]
    nanvaluepercentagearray = np.array(nanvaluepercentage)[idx]
    featuresarray = np.array(features)[idx]
    data = np.concatenate([nanvaluearray.reshape(1,-1),nanvaluepercentagearray.reshape(1,-1)],axis = 0)
    missingvaluedf = pd.DataFrame(data = data, columns = featuresarray, 
                                  index = ['Missing Value Number','Missing Value Percentage'])
    return missingvaluedf
    


In [10]:
features = df.columns.tolist()
nanvaluenumber,nanvaluepercentage,featureshavingnan = missingvaluepercentages(features,df)

In [11]:
missingvaldf = visualizemissingdata(nanvaluenumber,nanvaluepercentage,featureshavingnan)

In [12]:
display(missingvaldf.style.background_gradient(cmap = 'Reds',axis = 1))

,PEDROWNOTGRNT,EXCEPTRSNDESC,SPEEDING,INATTENTIONIND,INTKEY,EXCEPTRSNCODE,SDOTCOLNUM,JUNCTIONTYPE,X,Y,LIGHTCOND,WEATHER,ROADCOND,COLLISIONTYPE,ST_COLDESC,UNDERINFL,LOCATION,ADDRTYPE,ST_COLCODE,STATUS,OBJECTID,SEVERITYDESC,HITPARKEDCAR,INCDATE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,CROSSWALKKEY,INCDTTM,SDOT_COLCODE,SDOT_COLDESC,SEGLANEKEY,SEVERITYCODE
Missing Value Number,190006,189035,185340,164868,129603,109862,79737,6329,5334,5334,5170,5081,5012,4904,4904,4884,2677,1926,18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Missing Value Percentage,97.6,97.1,95.21,84.69,66.57,56.43,40.96,3.25,2.74,2.74,2.66,2.61,2.57,2.52,2.52,2.51,1.38,0.99,0.01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


<h3> Analysis </h3>

98% of the data in column **PEDROWNOTGRNT** is missing and we are not in a position to speculate if the Pedestrian was grated the right of way given the lack of information. Furthermore, both columns **EXCEPTRSDESC** and **EXCEPTRSNCODE** only have Not Enough Information as it's only available data, hence this isn't enough to make any conclusive action on the missing data of these columns. 

**INTKEY** is a key to each intersection. While intersections can have experience more collisions, which will be investigated later using box plots and barcharts, this column which already has so much of NaN values, can be removed.

**INATTENTIONIND** also has a large number of missing values. While we could replace the missing values with 'N', given the number of missing values, it would be an unnecessary amount of speculation. Hence we drop this column too. 

**SDOTCOLNUM** is only a reference number given to each collision, and has no impact on our prediction model. Hence we may remove this column too. 

In [13]:
df.drop(['PEDROWNOTGRNT','EXCEPTRSNDESC','EXCEPTRSNCODE','INTKEY','INATTENTIONIND','SDOTCOLNUM'],axis = 1,inplace = True)

In [14]:
#visualizing missing data after dropping the above columns
features = df.columns.tolist()
nanvaluenumber,nanvaluepercentage,featureshavingnan = missingvaluepercentages(features,df)
missingvaldf = visualizemissingdata(nanvaluenumber,nanvaluepercentage,featureshavingnan)
display(missingvaldf.style.background_gradient(cmap = 'Reds',axis = 1))

,SPEEDING,JUNCTIONTYPE,Y,X,LIGHTCOND,WEATHER,ROADCOND,COLLISIONTYPE,ST_COLDESC,UNDERINFL,LOCATION,ADDRTYPE,ST_COLCODE,PERSONCOUNT,OBJECTID,STATUS,SEVERITYDESC,HITPARKEDCAR,INCDATE,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,CROSSWALKKEY,INCDTTM,SDOT_COLCODE,SDOT_COLDESC,SEGLANEKEY,SEVERITYCODE
Missing Value Number,185340,6329,5334,5334,5170,5081,5012,4904,4904,4884,2677,1926,18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Missing Value Percentage,95.21,3.25,2.74,2.74,2.66,2.61,2.57,2.52,2.52,2.51,1.38,0.99,0.01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


<h3> Analysis </h3>
Now the main culprit is from the **SPEEDING** column, however it is safe to assume that while Y represents cases where speeding was involved, the missing values represent cases where speeding was not involved

In [15]:
#replacing 'Y' with 1 and Missing Values with 0 in the SPEEDING Column
df.loc[(df['SPEEDING'] == 'Y'),'SPEEDING'] = 1
df.loc[(df['SPEEDING'].isna()),'SPEEDING'] = 0
df['SPEEDING'].value_counts()

0    185340
1      9333
Name: SPEEDING, dtype: int64

In [16]:
#visualizing missing data after adjusting the speeding column
features = df.columns.tolist()
nanvaluenumber,nanvaluepercentage,featureshavingnan = missingvaluepercentages(features,df)
missingvaldf = visualizemissingdata(nanvaluenumber,nanvaluepercentage,featureshavingnan)
display(missingvaldf.style.background_gradient(cmap = 'Reds',axis = 1))

,JUNCTIONTYPE,X,Y,LIGHTCOND,WEATHER,ROADCOND,COLLISIONTYPE,ST_COLDESC,UNDERINFL,LOCATION,ADDRTYPE,ST_COLCODE,PERSONCOUNT,OBJECTID,STATUS,SEVERITYDESC,HITPARKEDCAR,INCDATE,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,CROSSWALKKEY,INCDTTM,SDOT_COLCODE,SDOT_COLDESC,SPEEDING,SEGLANEKEY,SEVERITYCODE
Missing Value Number,6329,5334,5334,5170,5081,5012,4904,4904,4884,2677,1926,18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Missing Value Percentage,3.25,2.74,2.74,2.66,2.61,2.57,2.52,2.52,2.51,1.38,0.99,0.01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [17]:
#drop the LOCATION column
df.drop(['LOCATION'],axis = 1,inplace = True)

In [18]:
df['UNDERINFL'].value_counts(dropna = False)

0.0    180668
1.0      9121
NaN      4884
Name: UNDERINFL, dtype: int64

<h3> Analysis </h3>

We shall drop all the NaN values off these columns. Note that **Unknown** and **Other** instances are still present in these columns. 


In [19]:
df.dropna(inplace = True)
df.reset_index(drop = True, inplace = True)
#df['SDOT_COLCODE'].replace(to_replace = {0:np.nan},inplace = True)
#df['SDOT_COLDESC'].replace(to_replace = {'NOT ENOUGH INFORMATION / NOT APPLICABLE':np.nan},inplace = True)

In [20]:
#visualizing missing data after replacing the Unknown and Others values
features = df.columns.tolist()
nanvaluenumber,nanvaluepercentage,featureshavingnan = missingvaluepercentages(features,df)
missingvaldf = visualizemissingdata(nanvaluenumber,nanvaluepercentage,featureshavingnan)
display(missingvaldf.style.background_gradient(cmap = 'Reds',axis = 1))

,HITPARKEDCAR,INCDATE,X,Y,OBJECTID,STATUS,ADDRTYPE,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INCDTTM,CROSSWALKKEY,JUNCTIONTYPE,SDOT_COLCODE,SDOT_COLDESC,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,SEVERITYCODE
Missing Value Number,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Missing Value Percentage,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Collision codes are worth investigating into to see if certain collisions have effect on the severity. Hence while we drop the description column of each collision, we keep the collision code column. 

In [21]:
st_colcode = df[['ST_COLCODE','ST_COLDESC']]
st_colcodefinal = st_colcode[~(st_colcode.duplicated())].style.hide_index()
sdot_colcode = df[['SDOT_COLCODE','SDOT_COLDESC']]
sdot_colcodefinal = sdot_colcode[~(sdot_colcode.duplicated())].style.hide_index()

In [22]:
df.drop(['ST_COLDESC','SDOT_COLDESC'],axis = 1, inplace = True)

In [23]:
df.head()

,SEVERITYCODE,X,Y,OBJECTID,STATUS,ADDRTYPE,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INCDATE,INCDTTM,JUNCTIONTYPE,SDOT_COLCODE,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,SPEEDING,ST_COLCODE,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,Matched,Intersection,Injury Collision,Angles,2,0,0,2,2013-03-27 00:00:00+00:00,3/27/2013 2:54:00 PM,At Intersection (intersection related),11,0,Overcast,Wet,Daylight,0,10,0,0,N
1,1,-122.347294,47.647172,2,Matched,Block,Property Damage Only Collision,Sideswipe,2,0,0,2,2006-12-20 00:00:00+00:00,12/20/2006 6:55:00 PM,Mid-Block (not related to intersection),16,0,Raining,Wet,Dark - Street Lights On,0,11,0,0,N
2,1,-122.334540,47.607871,3,Matched,Block,Property Damage Only Collision,Parked Car,4,0,0,3,2004-11-18 00:00:00+00:00,11/18/2004 10:20:00 AM,Mid-Block (not related to intersection),14,0,Overcast,Dry,Daylight,0,32,0,0,N
3,1,-122.334803,47.604803,4,Matched,Block,Property Damage Only Collision,Other,3,0,0,3,2013-03-29 00:00:00+00:00,3/29/2013 9:26:00 AM,Mid-Block (not related to intersection),11,0,Clear,Dry,Daylight,0,23,0,0,N
4,2,-122.306426,47.545739,5,Matched,Intersection,Injury Collision,Angles,2,0,0,2,2004-01-28 00:00:00+00:00,1/28/2004 8:04:00 AM,At Intersection (intersection related),11,0,Raining,Wet,Daylight,0,10,0,0,N


In [24]:
df.isnull().sum()

SEVERITYCODE     0
X                0
Y                0
OBJECTID         0
STATUS           0
ADDRTYPE         0
SEVERITYDESC     0
COLLISIONTYPE    0
PERSONCOUNT      0
PEDCOUNT         0
PEDCYLCOUNT      0
VEHCOUNT         0
INCDATE          0
INCDTTM          0
JUNCTIONTYPE     0
SDOT_COLCODE     0
UNDERINFL        0
WEATHER          0
ROADCOND         0
LIGHTCOND        0
SPEEDING         0
ST_COLCODE       0
SEGLANEKEY       0
CROSSWALKKEY     0
HITPARKEDCAR     0
dtype: int64

<h3> Analysis </h3>

As some of the attributes still consist of the entry **Unknown**. In fact a large number of entries are Unknown, hence dropping these data would result in loss of a large number of valuable training samples. Hence these entries are replaced by the most occuring entry in these columns.

In [25]:
featureswithunknownentry = []
for column in df.columns.tolist():
    if len(df.loc[df[column] == 'Unknown']) > 0:
        featureswithunknownentry.append(column)


C:\Users\ASUS\Anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [26]:
featureswithunknownentry

['JUNCTIONTYPE', 'WEATHER', 'ROADCOND', 'LIGHTCOND']

In [27]:
for feature in featureswithunknownentry:
    topoccurence = df[feature].describe()[2]
    df.loc[df[feature] == 'Unknown',feature] = topoccurence

Now all **Unknown** entries have been replaced in all columns. 

In [28]:
#Drop OBJECTID and STATUS columns
df.drop(['OBJECTID','STATUS'],axis = 1, inplace = True)

In [29]:
#replacing the 'Y' and 'N' values of HITPARKEDCAR column with 1 and 0 respectively
df.loc[df['HITPARKEDCAR'] == 'Y','HITPARKEDCAR'] = 1
df.loc[df['HITPARKEDCAR'] == 'N','HITPARKEDCAR'] = 0
df.head()

,SEVERITYCODE,X,Y,ADDRTYPE,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INCDATE,INCDTTM,JUNCTIONTYPE,SDOT_COLCODE,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,SPEEDING,ST_COLCODE,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,Intersection,Injury Collision,Angles,2,0,0,2,2013-03-27 00:00:00+00:00,3/27/2013 2:54:00 PM,At Intersection (intersection related),11,0,Overcast,Wet,Daylight,0,10,0,0,0
1,1,-122.347294,47.647172,Block,Property Damage Only Collision,Sideswipe,2,0,0,2,2006-12-20 00:00:00+00:00,12/20/2006 6:55:00 PM,Mid-Block (not related to intersection),16,0,Raining,Wet,Dark - Street Lights On,0,11,0,0,0
2,1,-122.334540,47.607871,Block,Property Damage Only Collision,Parked Car,4,0,0,3,2004-11-18 00:00:00+00:00,11/18/2004 10:20:00 AM,Mid-Block (not related to intersection),14,0,Overcast,Dry,Daylight,0,32,0,0,0
3,1,-122.334803,47.604803,Block,Property Damage Only Collision,Other,3,0,0,3,2013-03-29 00:00:00+00:00,3/29/2013 9:26:00 AM,Mid-Block (not related to intersection),11,0,Clear,Dry,Daylight,0,23,0,0,0
4,2,-122.306426,47.545739,Intersection,Injury Collision,Angles,2,0,0,2,2004-01-28 00:00:00+00:00,1/28/2004 8:04:00 AM,At Intersection (intersection related),11,0,Raining,Wet,Daylight,0,10,0,0,0


In [30]:
severitycode = df[['SEVERITYCODE']]
dffeatures = df[df.columns.tolist()[1:]]

In [31]:
dffeatures['INCTIME'] = pd.to_datetime(dffeatures['INCDTTM'])

<h3> Analysis </h3> 

As we have successfully converted the **INCDTTM** columns to datetime format, now we can use it to create time periods to identify the periods at which accidents occur and if the time range of accident has any correlation with the severity code. We shall specify 6 time periods: *Late Night, Early Morning, Morning, Noon, Evening, Night* 

In some cases we do not have the time of incident. For these we specify it as *Other*


In [32]:
dffeatures['TIMEOFDAY'] = np.nan
dffeatures.loc[((dffeatures['INCTIME'].dt.hour >= 0) & (dffeatures['INCTIME'].dt.hour < 4)) & 
               ((dffeatures['INCTIME'].dt.minute != 0)),'TIMEOFDAY'] = 'Late Night'
dffeatures.loc[(dffeatures['INCTIME'].dt.hour >= 4) & (dffeatures['INCTIME'].dt.hour < 8),'TIMEOFDAY'] = 'Early Morning'
dffeatures.loc[(dffeatures['INCTIME'].dt.hour >= 8) & (dffeatures['INCTIME'].dt.hour < 12),'TIMEOFDAY'] = 'Morning'
dffeatures.loc[(dffeatures['INCTIME'].dt.hour >= 12) & (dffeatures['INCTIME'].dt.hour < 16),'TIMEOFDAY'] = 'Noon'
dffeatures.loc[(dffeatures['INCTIME'].dt.hour >= 16) & (dffeatures['INCTIME'].dt.hour < 20),'TIMEOFDAY'] = 'Evening'
dffeatures.loc[(dffeatures['INCTIME'].dt.hour >= 20) & (dffeatures['INCTIME'].dt.hour < 24),'TIMEOFDAY'] = 'Night'
dffeatures.loc[(dffeatures['INCTIME'].dt.hour == 0) & (dffeatures['INCTIME'].dt.minute == 0) &
               (dffeatures['INCTIME'].dt.second == 0),'TIMEOFDAY'] = 'Other'

In [33]:
dffeatures['YEAR'] = dffeatures['INCDATE'].dt.year
dffeatures['MONTH'] = dffeatures['INCDATE'].dt.month
dffeatures['DAY'] = dffeatures['INCDATE'].dt.dayofweek

Day of the week format in Python is as follows:
 - 0 : Monday
 - 1 : Tuesday
 - 2 : Wednesday
 - 3 : Thursday
 - 4 : Friday
 - 5 : Saturday
 - 6 : Sunday